In [ ]:
# Prompt 1
# how to load pyspark delta table to pnr_df dataframe 
pnr_df = spark.read.format("delta").load("path/to/delta")

In [ ]:
#Prompt 2 
#create a new dataframe joined_df by Joining pnr_df and distance_master_df on following conditionsl (ORIG_AP_CD = airport_origin_code AND DESTN_AP_CD = airport_destination_code) OR (ORIG_AP_CD = airport_destination_code AND DESTN_AP_CD = airport_origin_code

from pyspark.sql.functions import col
    Joined_df = pnr_df.join(distance_master_df, (
        ((col("ORIG_AP_CD") == col("airport_origin_code")) & (col("DESTN_AP_CD") == col("airport_destination_code"))) |
        ((col("ORIG_AP_CD") == col("airport_destination_code")) & (col("DESTN_AP_CD") == col("airport_origin_code"))))
        , how="left")

In [ ]:
# prompt 3
# create a new dataframe joined_df by Joining Joined_df and location_master_df on following conditionsl (ORIG_AP_CD = ap_cd_val) OR (DESTN_AP_CD = ap_cd_val)
Joined_df = Joined_df.join(location_master_df, col("ORIG_AP_CD") == col("ap_cd_val"), how="left")


In [ ]:
# prompt 4 
# rename the columns ap_cd_val, latitude and longitude to ap_cd_val_origin, latitude_origin and longitude_origin

newJoined_df=Joined_df.withColumnRenamed("ap_cd_val","ap_cd_val_origin") \
.withColumnRenamed("latitude","latitude_origin") \
.withColumnRenamed("longitude","longitude_origin")

In [ ]:
# prompt 5
# rename the columns ap_cd_val, latitude and longitude to ap_cd_val_destination, latitude_destination and longitude_destination

new_Joined_df = new_Joined_df.join(location_master_df, col("DESTN_AP_CD") == col("ap_cd_val"), how="left")

In [ ]:
# prompt 6
# rename the columns ap_cd_val, latitude and longitude to ap_cd_val_desti, latitude_desti and longitude_desti
# rename the column distance_in_km to distance_in_km_OD
# rename the column id to id_OD

new_Joined_df = new_Joined_df.withColumnRenamed("ap_cd_val", "ap_cd_val_desti") \
    .withColumnRenamed("latitude", "latitude_desti") \
    .withColumnRenamed("longitude", "longitude_desti") \
    .withColumnRenamed("distance_in_km", "distance_in_km_OD") \
    .withColumnRenamed("id", "id_OD")

In [ ]:
# prompt 7
# Create a Pyspark UDF to convert DMS to Decimal degress ( copilot prompt 7)
# This udf should take one parameter DMS, it should calculates the decimal degrees based on degrees, minutes, seconds, direction values and returns the result. If the direction is either 'S' or 'W', it negates the result because these directions represent negative values in the coordinate system.


from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

def dms_to_decimal(degrees, minutes, seconds, direction):
    decimal_degrees = degrees + (minutes / 60) + (seconds / 3600)
    if direction in ['S', 'W']:
        decimal_degrees = -decimal_degrees
    return decimal_degrees

dms_to_decimal_udf = udf(dms_to_decimal, DoubleType())

In [ ]:

#prompt 8
#Apply the UDF to all the latitude and longitude columns

from pyspark.sql.functions import col

# Apply the UDF to the latitude and longitude columns
new_Joined_df = new_Joined_df.withColumn("latitude_origin", dms_to_decimal_udf(col("latitude_origin")))
new_Joined_df = new_Joined_df.withColumn("longitude_origin", dms_to_decimal_udf(col("longitude_origin")))
new_Joined_df = new_Joined_df.withColumn("latitude_desti", dms_to_decimal_udf(col("latitude_desti")))
new_Joined_df = new_Joined_df.withColumn("longitude_desti", dms_to_decimal_udf(col("longitude_desti")))

In [ ]:
# prompt 9
# Write a Python UDF to calculate the distance between two points using their latitude and longitude. The UDF accept the latitude and longitude for both points and return the distance in kilometers using the Haversine formula.

import math

def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = 6371 * c  # Radius of the Earth in kilometers

    return distance